In [1]:
import numpy as np
import pandas as pd

import arviz as az

import bebi103

import bokeh.io
import bokeh.plotting
bokeh.io.output_notebook()

/home/ec2-user/miniconda/lib/python3.7/site-packages/bebi103/viz.py:30: UserWarning: DataShader import failed with error "No module named 'datashader'".
Features requiring DataShader will not work and you will get exceptions.
  Features requiring DataShader will not work and you will get exceptions.""")


Loading BokehJS ...

## Problem 8.2: Microtubule catastrophe, 40 pts

_Note: This problem is best done after the lecture November 22._

In this problem, we use data from [Gardner, Zanic, et al., Depolymerizing kinesins Kip3 and MCAK shape cellular microtubule architecture by differential control of catastrophe, *Cell*, **147**, 1092-1103, 2011](https://doi.org/10.1016/j.cell.2011.10.037). The authors investigated the dynamics of microtubule catastrophe, the switching of a microtubule from a growing to a shrinking state.  In particular, they were interested in the time between the start of growth of a microtubule and the catastrophe event. They monitored microtubules in a single-molecule [TIRF assay](https://en.wikipedia.org/wiki/Total_internal_reflection_fluorescence_microscope) by using tubulin (the monomer that comprises a microtubule) that was labeled with a fluorescent marker. As a control to make sure that fluorescent labels and exposure to laser light did not affect the microtubule dynamics, they performed a similar experiment using differential interference contrast (DIC) microscopy. They measured the time until catastrophe with labeled and unlabeled tubulin. We will carefully analyze the data and make some conclusions about the processes underlying microtubule catastrophe.

In the file `gardner_mt_catastrophe_only_tubulin.csv` (which you can download [here](../data/gardner_mt_catastrophe_only_tubulin.csv)), we have observed catastrophe times of microtubules with different concentrations of tubulin. To start with, we will consider the experiment run with a tubulin concentration of 12 µM. So, our data set consists of a set of measurements of the amount of time to catastrophe. We will consider three models for microtubule catastrophe.

- Model 1: The time to catastrophe is Exponentially distributed.
- Model 2: The time to catastrophe is Gamma distributed.
- Model 3: The time to catastrophe is Weibull distributed.

Note that these descriptions are for the likelihood; we have not specified priors.


**a)  Describe the three models in words. Give physical descriptions of the meanings of their parameters. Describe how these models are related to each other. Tutorial 3c will be useful.** 

<br />



- Model 1: The time to catastrophe is Exponentially distributed.

This suggests that the occurance of catastrophe is a Poisson process, so it is a "rare event" that requires multiple subprocesses to lead it it. The parameter for the process, if it is exponential, $\beta$, represents the characteristic rate of catastrophe, that is how often catastrophe happens in a certain amount of time. It can also be parametrized as $\tau=1/\beta$, the characteristic catastrophe time, which fits what we are given in our data. The Exponential distribution is a special case of the Gamma distribution where $\alpha = 1$ and a special case of the Weibull distribution where $\alpha = 1$ and $\sigma=1/\beta$



- Model 2: The time to catastrophe is Gamma distributed.

This suggests that the occurance of catastrophe represents a specific number of occurances of a Poisson process, that is a discrete number of steps that occur at the same rate must occur for catastrophe to occur. There are two parameters for this distribution, $\alpha$ and $\beta$, where $\alpha$ is the number of arrivals (or "steps") required to trigger catastrophe, and $\beta$ is the rate of the arrivals. Thus, the characteristic catastrophe time is given by $\alpha/\beta$. 


- Model 3: The time to catastrophe is Weibull distributed.

This suggests that the likelihood of catastrophe is dependent on the amount of time it has been since the last catastrophe, so the longer it has been since the last catastrophe, the more likely it is that catastrophe will occur. There are two parameters for this distribution, $\alpha$ which defines how the probability changes over time, and $\sigma$ which is the characteristic catastrophe time.

**b) Perform parameter estimates for the respective models and make model comparisons. Comment on what this means with respect to our understanding of how microtubule catastrophe works.**

We first load in our data and come up with our priors for the parameters of the three distributions. We don't have much prior knowledge so we will keep them simple (normal distribution).

*Exponential(tao)*

tao ~ normal(700, 100)

*Gamma($\alpha$, $\beta$)*

alpha ~ normal(10, 3)

beta ~ normal(10, 3)

*Weibull($\alpha$, $\sigma$)*

$\alpha$ ~ normal(3, 0.05)

$\sigma$ ~ normal(10, 3)


In [2]:
df = pd.read_csv('/home/ec2-user/data/gardner_mt_catastrophe_only_tubulin.csv', comment = "#")

In [91]:
prior_sm1 = bebi103.stan.StanModel(file='./8.2_prior_pred_12_m1.stan')
prior_sm2 = bebi103.stan.StanModel(file='./8.2_prior_pred_12_m2.stan')
prior_sm3 = bebi103.stan.StanModel(file='./8.2_prior_pred_12_m3.stan')

Using cached StanModel.
Using cached StanModel.
Using cached StanModel.


In [79]:
# Store input parameters in a dictionary so stan can access them
data = dict(N=692)

# Generate samples
samples_gen1 = prior_sm1.sampling(data=data,
                          algorithm='Fixed_param',
                          warmup=0,
                          chains=1,
                          iter=1000)
samples_gen2 = prior_sm2.sampling(data=data,
                          algorithm='Fixed_param',
                          warmup=0,
                          chains=1,
                          iter=1000)
samples_gen3 = prior_sm3.sampling(data=data,
                          algorithm='Fixed_param',
                          warmup=0,
                          chains=1,
                          iter=1000)

# Store samples in a dataframe
df_gen1 = bebi103.stan.to_dataframe(samples_gen1, diagnostics=False)
df_gen2 = bebi103.stan.to_dataframe(samples_gen2, diagnostics=False)
df_gen3 = bebi103.stan.to_dataframe(samples_gen3, diagnostics=False)

# Let's look at one of the dataframes to make sure they look away
df_gen3.head()

,chain,chain_idx,warmup,uM_12[1],uM_12[2],uM_12[3],uM_12[4],uM_12[5],uM_12[6],uM_12[7],...,uM_12[686],uM_12[687],uM_12[688],uM_12[689],uM_12[690],uM_12[691],uM_12[692],alpha,sigma,lp__
0,1,1,0,2.959655,3.488298,3.201515,4.216934,1.911634,2.140977,3.628671,...,4.151919,6.052353,5.936146,5.241189,6.441932,5.854771,3.107409,3.020094,4.406701,0.0
1,1,2,0,1.982123,1.200279,2.774964,2.355275,2.440783,3.445144,3.675388,...,3.647276,3.920916,2.906405,2.223397,3.888068,2.752943,2.243082,3.024492,3.156088,0.0
2,1,3,0,7.700769,6.654504,5.661729,8.617962,12.990552,10.214778,7.379447,...,5.204354,7.743165,10.757210,15.823748,8.153561,8.860838,10.206375,3.009141,9.028180,0.0
3,1,4,0,7.472062,12.044283,5.211900,8.707415,7.878539,13.799459,20.541731,...,9.460223,15.048061,11.135417,5.804754,16.723802,12.323126,14.003778,2.941747,14.330821,0.0
4,1,5,0,2.985921,1.987467,1.407783,3.967241,3.972194,1.150371,1.955392,...,6.143686,5.605729,4.971060,4.336268,3.420215,2.667542,5.791918,3.069584,3.704991,0.0


In [90]:
p = bebi103.viz.predictive_ecdf(samples_gen1, "uM_12",
                                x_axis_label = "intercatastrophe time (s)")
p.x_range = bokeh.models.Range1d(-10, 10000)
bokeh.io.show(p)

In [85]:
p = bebi103.viz.predictive_ecdf(samples_gen2, "uM_12",
                                x_axis_label = "intercatastrophe time (s)")
p.x_range = bokeh.models.Range1d(-10, 10)
bokeh.io.show(p)

In [81]:
p = bebi103.viz.predictive_ecdf(samples_gen3, "uM_12",
                                x_axis_label = "intercatastrophe time (s)")
p.x_range = bokeh.models.Range1d(-10, 50)
bokeh.io.show(p)

Our priors look reasonable, so now we can move on to creating our mcmc models and sampling.

In [67]:
sm1 = bebi103.stan.StanModel(file='./8.2_mcmc_12_m1.stan')
sm2 = bebi103.stan.StanModel(file='./8.2_mcmc_12_m2.stan')
sm3 = bebi103.stan.StanModel(file='./8.2_mcmc_12_m3.stan')

Using cached StanModel.
Using cached StanModel.
Using cached StanModel.


In [51]:
data = dict(N=len(df),
           uM_12=df['12 uM'].values.astype(float))

In [60]:
samples1 = sm1.sampling(data=data)
samples2 = sm2.sampling(data=data)
samples3 = sm3.sampling(data=data)

In [61]:
df_mcmc1 = bebi103.stan.to_dataframe(samples1, diagnostics=False, inc_warmup=False)
df_mcmc2 = bebi103.stan.to_dataframe(samples2, diagnostics=False, inc_warmup=False)
df_mcmc3 = bebi103.stan.to_dataframe(samples3, diagnostics=False, inc_warmup=False)

In [62]:
df_mcmc3.head()

,chain,chain_idx,warmup,alpha,sigma,log_like[1],log_like[2],log_like[3],log_like[4],log_like[5],...,uM_12_ppc[684],uM_12_ppc[685],uM_12_ppc[686],uM_12_ppc[687],uM_12_ppc[688],uM_12_ppc[689],uM_12_ppc[690],uM_12_ppc[691],uM_12_ppc[692],lp__
0,1,1,0,0.665520,80.554756,-4.863764,-5.189537,-5.189537,-5.287579,-5.364648,...,198.620163,11.098617,69.380277,0.320216,49.748341,2.319930,386.777988,6.400399,0.284556,-6875.511834
1,1,2,0,0.648505,85.074963,-4.898102,-5.224358,-5.224358,-5.321834,-5.398238,...,132.558162,29.099731,16.968297,154.637017,1.565568,17.459336,32.636257,42.508185,0.544517,-6872.636984
2,1,3,0,0.682327,84.213002,-4.866421,-5.180813,-5.180813,-5.275832,-5.350657,...,179.780255,96.451287,10.179080,47.230644,79.154152,227.621295,1.912042,216.915176,16.025026,-6871.143061
3,1,4,0,0.677586,84.760981,-4.872626,-5.188130,-5.188130,-5.283310,-5.358206,...,207.268492,63.782835,400.182804,674.366415,1.861013,286.322047,23.228846,5.646027,11.912947,-6870.308615
4,1,5,0,0.657500,85.222696,-4.891023,-5.213671,-5.213671,-5.310342,-5.386202,...,272.499014,345.552418,82.564120,162.514377,54.266604,0.683849,5.648467,32.668135,131.112216,-6871.198522


In [65]:
bebi103.stan.compare({'exponential': samples1, 'gamma': samples2, 'weibull': samples3},
                     log_likelihood='log_like', ic='loo')

,loo,ploo,dloo,weight,se,dse,warning
gamma,9278.44,2.14642,0,0.984134,45.7341,0,0
exponential,9608.84,0.364711,330.406,0.0158657,31.3845,28.7602,0
weibull,10911.6,1.38093,1633.15,1.23517e-09,68.3094,51.0139,0


**c)** Using whichever model you favor based on your work in part (b), obtain parameter estimates for the other tubulin concentrations. Given that microtubules polymerize faster with higher tubulin concentrations, is there anything you can say about the occurrence of catastrophe by looking at the values of the parameters versus tubulin concentration?
